TODO:
- cruzar regras de atendimento parcial com metas do mes (ating/pont/acum)
- acum pode significar ating+pontos
- transformar em regex

# Ambev data challenge
## Adriano Freitas

## Modelo de previsão de cumprimento da meta

Este notebook tem o objetivo de criar um modelo para prever o cumprimento da meta.

In [1]:
%%capture

%run ./00-agf-utils.ipynb

%store -r default_color
%store -r default_light_color
%store -r default_dark_color
%store -r colormap
%store -r figsize

In [2]:
data_path = '../data/processed/'
file_name = 'ambev-final-dataset-processed.csv'

In [3]:
df = pd.read_csv(os.path.join(data_path, file_name))
df.shape
df.head()

/Users/adrianofreitas/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(106217, 32)

,ord_mes_referencia,nom_pais,nom_mundo,dis_regional_area,dis_unidade,nom_grupo_cargo,nom_cargo,dis_grade,nom_banda,nom_area,dis_nome_funcionario,dis_nome_gestor,nom_codigo_kpi,nom_diretoria,nom_areas_diretoria,nom_funcao,nom_tipo_meta,nom_categoria_kpi,dis_nome_kpi,per_peso_kpi,nom_prazo,nom_regra_alcance_parcial,bin_meta_projeto,per_ating_mes,per_pontos_mes,per_acum_mes,per_ating_acumulado,per_pontos_acumulado,per_acum_acumulado,per_ating_fim_exer,per_pontos_fim_exer,per_acum_fim_exer
0,3.00,Brasil,Supply,2.00,192.00,ANALISTA,ANALISTA III,10.00,VII-A,CENG,409.00,934.00,001CC0069,Diretoria Industrial,CENG/CDT,ANALISTA,Mandatório,Cash Flow,1990.00,20.00,12/31/2017 12:00:00 AM,100 % = 5 KPI?s 90% = 4 KPI?s80% = 3 KPI?s,0.00,100.00,100.00,20.00,100.00,100.00,20.00,100.00,100.00,20.00
1,3.00,Brasil,Supply,2.00,192.00,ANALISTA,ANALISTA II,9.00,VII-B,CENG,1537.00,1640.00,001CC0069,Diretoria Industrial,CENG/CDT,ANALISTA,Mandatório,Cash Flow,1990.00,20.00,12/31/2017 12:00:00 AM,100 % = 5 KPI?s 90% = 4 KPI?s80% = 3 KPI?s,0.00,80.00,60.00,12.00,80.00,60.00,12.00,100.00,100.00,20.00
2,3.00,Brasil,Supply,2.00,192.00,ANALISTA,ANALISTA IV,11.00,VI-C,CENG,1681.00,934.00,001CC0069,Diretoria Industrial,CENG/CDT,ANALISTA,Mandatório,Cash Flow,1990.00,20.00,12/31/2017 12:00:00 AM,100 % = 5 KPI?s 90% = 4 KPI?s80% = 3 KPI?s,0.00,100.00,100.00,20.00,100.00,100.00,20.00,100.00,100.00,20.00
3,3.00,Brasil,Supply,2.00,192.00,ANALISTA,ANALISTA III,10.00,VII-A,CENG,2007.00,934.00,001CC0069,Diretoria Industrial,CENG/CDT,ANALISTA,Mandatório,Cash Flow,1990.00,20.00,12/31/2017 12:00:00 AM,100 % = 5 KPI?s 90% = 4 KPI?s80% = 3 KPI?s,0.00,100.00,100.00,20.00,100.00,100.00,20.00,100.00,100.00,20.00
4,3.00,Brasil,Supply,2.00,192.00,ANALISTA,ANALISTA IV,11.00,VI-C,CENG,2445.00,293.00,001CC0069,Diretoria Industrial,CENG/CDT,ANALISTA,Mandatório,Cash Flow,1990.00,20.00,12/31/2017 12:00:00 AM,100 % = 5 KPI?s 90% = 4 KPI?s80% = 3 KPI?s,0.00,100.00,100.00,20.00,100.00,100.00,20.00,100.00,100.00,20.00


## Definição do target
Baseado no entendimento do dataset, temos 3 tipos de variáveis que retratam o atingimento das metas, são eles:
- **Atingido (ating)**: Qual é o percentual da meta atingido no mês.
- **Pontos (pontos)**: Os pontos são calculados baseado na regra de atingimento parcial (coluna `nom_regra_alcance_parcial`). Este valor é utilizado para calcular o atingimento final da meta.
- **Acumulativo (acum)**: Mostra de forma acumulativa decompondo a meta pelo peso do KPI. Esta coluna é calculada aplicando o peso do kpi sobre os pontos.

Decidimos então prever o percentual da meta que o funcionário vai atingir no mês, uma vez que essa é a medida raiz, sendo as outras colunas calculadas derivadas dessa. Com essa previsão poderemos calcular as demais e chegar na previsão do final do exercício.

## Definição do modelo

Decidimos aplicar uma rede neural recorrente (RNN) com células LSTM (Long Short Term Memory), que possuem ótima performance em séries temporais como essa em questão.

Definimos um intervalo de observação de 3 meses, o que nos dará uma visão de sazonalidade. Nos baseamos na duração de cada estação do ano. Como temos um número de observações diferente a cada mês, vamos usar a média de observações de cada mês multiplicado por 3 como intervalo.

A média encontrada foi de 10621 registros.

In [4]:
# definindo quantos registros tem em média por mês
# essa será nossa janela uma vez que cada mês possui uma quantidade diferente
# de registros, isso nos dará aproximadamente 3 meses de observações
# para gerar um previsão
df.groupby(by='ord_mes_referencia').count().mean()

nom_pais                    10621.70
nom_mundo                   10616.50
dis_regional_area           10621.70
dis_unidade                 10621.70
nom_grupo_cargo             10621.70
nom_cargo                   10621.70
dis_grade                   10621.70
nom_banda                   10621.70
nom_area                    10616.50
dis_nome_funcionario        10621.70
dis_nome_gestor             10621.70
nom_codigo_kpi              10621.70
nom_diretoria               10621.70
nom_areas_diretoria         10621.70
nom_funcao                  10621.70
nom_tipo_meta               10621.70
nom_categoria_kpi           10621.70
dis_nome_kpi                10621.70
per_peso_kpi                10621.70
nom_prazo                   10621.70
nom_regra_alcance_parcial    9582.40
bin_meta_projeto            10621.70
per_ating_mes               10621.70
per_pontos_mes              10621.70
per_acum_mes                10621.70
per_ating_acumulado         10621.70
per_pontos_acumulado        10621.70
p

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

/Users/adrianofreitas/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Preparando dataframe

#### Scaling e Encoding

In [6]:
pattern = re.compile('nom_\w+')
col_search = np.vectorize(lambda x, pattern=pattern: bool(pattern.search(x)))
idx_filter = col_search(df.columns)
nom_cols = df.columns[idx_filter]

In [7]:
drop_cols = [
    'ord_mes_referencia', 
    'dis_nome_kpi', 
    'per_peso_kpi', 
    'nom_prazo',
    'nom_regra_alcance_parcial', 
    'bin_meta_projeto', 
    'per_pontos_mes',
    'per_acum_mes', 
    'per_ating_acumulado',
    'per_pontos_acumulado', 
    'per_acum_acumulado', 
    'per_ating_fim_exer',
    'per_pontos_fim_exer', 
    'per_acum_fim_exer'
]

prep_df = Prep(df) \
    .drop_cols(drop_cols) \
    .encode(nom_cols) \
    .scale()

In [8]:
prep_df.df.head()

,nom_pais,nom_mundo,dis_regional_area,dis_unidade,nom_grupo_cargo,nom_cargo,dis_grade,nom_banda,nom_area,dis_nome_funcionario,dis_nome_gestor,nom_codigo_kpi,nom_diretoria,nom_areas_diretoria,nom_funcao,nom_tipo_meta,nom_categoria_kpi,per_ating_mes
0,0.00,1.00,0.09,0.58,0.02,0.04,0.40,0.67,0.19,0.05,0.47,0.00,0.15,0.14,0.02,0.67,0.00,1.00
1,0.00,1.00,0.09,0.58,0.02,0.03,0.30,0.78,0.19,0.20,0.82,0.00,0.15,0.14,0.02,0.67,0.00,0.80
2,0.00,1.00,0.09,0.58,0.02,0.04,0.50,0.56,0.19,0.21,0.47,0.00,0.15,0.14,0.02,0.67,0.00,1.00
3,0.00,1.00,0.09,0.58,0.02,0.04,0.40,0.67,0.19,0.26,0.47,0.00,0.15,0.14,0.02,0.67,0.00,1.00
4,0.00,1.00,0.09,0.58,0.02,0.04,0.50,0.56,0.19,0.31,0.15,0.00,0.15,0.14,0.02,0.67,0.00,1.00


#### Spliting X e y mantendo sequência de 3 meses

In [ ]:
train_df = prep_df.df.values
X, y = [], []
i_len = 10621 * 3

for i in range (i_len, len(train_df)):
    X.append(train_df[i - i_len : i, :17])
    y.append(train_df[i, 17])
    
X, y = np.array(X), np.array(y)
print(X.shape, y.shape)

## Criando o modelo

In [ ]:
input_shape = (X.shape[1], 17)

model = Sequential()
model.add(LSTM(units = 100, 
               return_sequences = True, 
               input_shape = input_shape))
model.add(Dropout(0.3))

model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.3))

model.add(LSTM(units = 50))
model.add(Dropout(0.3))

model.add(Dense(units = 1, activation = 'sigmoid'))  # linear

model.compile(optimizer='rmsprop', 
              loss='mean_squared_error', 
              metrics=['mean_absolute_error'])

es = EarlyStopping(monitor='loss', min_delta=1e-10, patience=10, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, verbose=1)
mcp = ModelCheckpoint(filepath='weights.{epoch:02d}-{val_loss:.2f}.hdf5', 
                      monitor='loss', save_best_only=True, verbose=1)

model.fit(X, y, epochs=100, batch_size=32, callbacks=[es, reduce_lr, mcp])